In [1]:
import h5py
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.layers import *
from tensorflow import keras
from tensorflow.keras import layers, models, Model
from sklearn.metrics import roc_curve, auc
import tensorflow.keras.backend as K
import qkeras
from qkeras import *
import hls4ml

np.random.seed(42)
tf.random.set_seed(42)

(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()
X_train = np.reshape(X_train[:, :10, :10], (X_train.shape[0], 10, 10, 1)) / 255.
X_test = np.reshape(X_test[:, :10, :10], (X_test.shape[0], 10, 10, 1)) / 255.

def select_classes(x, y, classes):
    idx = (y == 99)
    for i in range(len(classes)):
        idx |= (y == classes[i])
    x, y = x[idx], y[idx]
    return x, y

classes = [0,1,2,3,4,5,6,7,8,9]
X_train, Y_train = select_classes(X_train, Y_train, classes)
X_test, Y_test = select_classes(X_test, Y_test, classes)

LE = LabelEncoder()
Y_train = LE.fit_transform(Y_train)
Y_test = LE.fit_transform(Y_test)

Y_train = to_categorical(Y_train, len(classes))
Y_test = to_categorical(Y_test, len(classes))

#X_train = np.reshape(X_train, (X_train.shape[0], -1))
#X_test = np.reshape(X_test, (X_test.shape[0], -1))

print("X_train.shape: " + str(X_train.shape))
print("Y_train.shape: " + str(Y_train.shape))
print("X_test.shape: " + str(X_test.shape))
print("Y_test.shape: " + str(Y_test.shape))

WARN: Unable to import optimizer(s) from expr_templates.py: No module named 'sympy'
X_train.shape: (60000, 10, 10, 1)
Y_train.shape: (60000, 10)
X_test.shape: (10000, 10, 10, 1)
Y_test.shape: (10000, 10)


/Users/hofungtsoi/miniforge3/envs/sparse/lib/python3.10/site-packages/hls4ml/converters/__init__.py:28: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


In [2]:
quantizer = quantized_bits(16, 6, alpha=1)
quantized_relu = 'quantized_relu(16, 6)'

x_in = keras.Input(shape=(50, 40, 1), name='x_in')

x = QConv2D(filters=1, kernel_size=(3, 3), use_bias=False, name='conv1',
            padding='same', strides=1,
            kernel_quantizer=quantizer, bias_quantizer=quantizer)(x_in)
x = QActivation(quantized_relu, name='relu1')(x)
x = layers.AveragePooling2D(2, name='pool1')(x)

x = QConv2D(filters=1, kernel_size=(3, 3), use_bias=False, name='conv2',
            padding='same', strides=1,
            kernel_quantizer=quantizer, bias_quantizer=quantizer)(x)
x = QActivation(quantized_relu, name='relu2')(x)
x = layers.AveragePooling2D(5, name='pool2')(x)

x = layers.Flatten(name='flatten')(x)

x = QDense(1, kernel_quantizer=quantizer, bias_quantizer=quantizer, name='dense1')(x)
    
#x = QConv2D(filters=1, kernel_size=(3, 3), use_bias=False, name='conv2',
#            padding='same', strides=1,
#            kernel_quantizer=quantizer, bias_quantizer=quantizer)(x)
#x = QActivation(quantized_relu, name='relu2')(x)

model = keras.Model(x_in, x, name='test_model')

model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.005),
              loss='mse', metrics = ['accuracy'])
model.summary()

Model: "test_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 x_in (InputLayer)           [(None, 50, 40, 1)]       0         
                                                                 
 conv1 (QConv2D)             (None, 50, 40, 1)         9         
                                                                 
 relu1 (QActivation)         (None, 50, 40, 1)         0         
                                                                 
 pool1 (AveragePooling2D)    (None, 25, 20, 1)         0         
                                                                 
 conv2 (QConv2D)             (None, 25, 20, 1)         9         
                                                                 
 relu2 (QActivation)         (None, 25, 20, 1)         0         
                                                                 
 pool2 (AveragePooling2D)    (None, 5, 4, 1)           0

In [ ]:
history = model.fit(X_train, X_train,
                    #validation_data = (X_val, X_val),
                    epochs=1, batch_size=128)

In [ ]:
plt.figure(figsize = (15,10))
axes = plt.subplot(2, 2, 1)
axes.plot(history.history['loss'], label = 'train loss')
axes.plot(history.history['val_loss'], label = 'val loss')
axes.legend(loc = "upper right")
axes.set_xlabel('Epoch')
axes.set_ylabel('Loss')

In [ ]:
Y_pred = model.predict(X_test)
print("Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_pred, axis=1))))

In [ ]:
def plot_roc(y_test, y_pred, labels):
    for x, label in enumerate(labels):        
        fpr, tpr, _ = roc_curve(y_test[:, x], y_pred[:, x])
        plt.plot(fpr, tpr, label='{0} tagger, AUC = {1:.1f}'.format(label, auc(fpr, tpr)*100.), linestyle='-')
    #plt.semilogy()
    #plt.semilogx()
    plt.ylabel("Signal Efficiency")
    plt.xlabel("Background Efficiency")
    #plt.ylim(0.00001, 1)
    #plt.xlim(0.00001, 1)
    plt.grid(True)
    plt.legend(loc='best')  
    
plt.figure(figsize=(4, 4))
plot_roc(Y_test, Y_pred, ['g','q','w','z','t'])

In [ ]:
model.save('model_test.h5')

In [ ]:
model = qkeras.utils.load_qmodel('model_test.h5')

In [ ]:
Y_pred = model.predict(X_test)

# HLS

In [3]:
#model = tf.keras.models.load_model('model-pointwisePhi.keras')
#model = tf.keras.models.load_model('model-densePhi.keras')

config = hls4ml.utils.config_from_keras_model(model, granularity='name')

config['Model']['Strategy'] = 'Latency'
config['LayerName']['x_in']['Precision'] = 'ap_fixed<12, 4, AP_RND, AP_SAT>'
config['LayerName']['conv1']['ParallelizationFactor'] = 20

config['LayerName']['conv2']['ParallelizationFactor'] = 5

# for dense Phi
if False:
    config['LayerName']['phi1']['ParallelizationFactor'] = 1
    config['LayerName']['phi1']['ReuseFactor'] = 8
    config['LayerName']['phi1']['Strategy'] = 'Latency'
    #config['LayerName']['phi1']['ConvImplementation'] = 'Pointwise'
    
    config['LayerName']['phi2']['ParallelizationFactor'] = 1
    config['LayerName']['phi2']['ReuseFactor'] = 8
    config['LayerName']['phi2']['Strategy'] = 'Latency'
    #config['LayerName']['phi2']['ConvImplementation'] = 'Pointwise'
    
    config['LayerName']['phi3']['ParallelizationFactor'] = 1
    config['LayerName']['phi3']['ReuseFactor'] = 8
    config['LayerName']['phi3']['Strategy'] = 'Latency'
    #config['LayerName']['phi3']['ConvImplementation'] = 'Pointwise'

config

Interpreting Model
Topology:
Layer name: x_in, layer type: InputLayer, input shapes: [[None, 50, 40, 1]], output shape: [None, 50, 40, 1]
Layer name: conv1, layer type: QConv2D, input shapes: [[None, 50, 40, 1]], output shape: [None, 50, 40, 1]
Layer name: relu1, layer type: Activation, input shapes: [[None, 50, 40, 1]], output shape: [None, 50, 40, 1]
Layer name: pool1, layer type: AveragePooling2D, input shapes: [[None, 50, 40, 1]], output shape: [None, 25, 20, 1]
Layer name: conv2, layer type: QConv2D, input shapes: [[None, 25, 20, 1]], output shape: [None, 25, 20, 1]
Layer name: relu2, layer type: Activation, input shapes: [[None, 25, 20, 1]], output shape: [None, 25, 20, 1]
Layer name: pool2, layer type: AveragePooling2D, input shapes: [[None, 25, 20, 1]], output shape: [None, 5, 4, 1]
Layer name: flatten, layer type: Reshape, input shapes: [[None, 5, 4, 1]], output shape: [None, 20]
Layer name: dense1, layer type: QDense, input shapes: [[None, 20]], output shape: [None, 1]


/Users/hofungtsoi/miniforge3/envs/sparse/lib/python3.10/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


{'Model': {'Precision': {'default': 'fixed<16,6>'},
  'ReuseFactor': 1,
  'Strategy': 'Latency',
  'BramFactor': 1000000000,
  'TraceOutput': False},
 'LayerName': {'x_in': {'Trace': False,
   'Precision': 'ap_fixed<12, 4, AP_RND, AP_SAT>'},
  'conv1': {'Trace': False,
   'Precision': {'result': 'auto',
    'weight': 'fixed<16,7,TRN,WRAP,0>',
    'bias': 'fixed<16,7,TRN,WRAP,0>'},
   'ParallelizationFactor': 20},
  'conv1_linear': {'Trace': False, 'Precision': {'result': 'auto'}},
  'relu1': {'Trace': False,
   'Precision': {'result': 'ufixed<16,6,RND_CONV,SAT,0>'}},
  'pool1': {'Trace': False, 'Precision': {'result': 'auto'}},
  'conv2': {'Trace': False,
   'Precision': {'result': 'auto',
    'weight': 'fixed<16,7,TRN,WRAP,0>',
    'bias': 'fixed<16,7,TRN,WRAP,0>'},
   'ParallelizationFactor': 5},
  'conv2_linear': {'Trace': False, 'Precision': {'result': 'auto'}},
  'relu2': {'Trace': False,
   'Precision': {'result': 'ufixed<16,6,RND_CONV,SAT,0>'}},
  'pool2': {'Trace': False, 'Prec

In [4]:
hls_model = hls4ml.converters.convert_from_keras_model(
    model,
    hls_config=config,
    project_name='model_test',
    output_dir='model_test',
    part='xcvu13p-flga2577-2-e',
    io_type='io_parallel',
)

hls_model.compile()
hls_model.write()

Interpreting Model
Topology:
Layer name: x_in, layer type: InputLayer, input shapes: [[None, 50, 40, 1]], output shape: [None, 50, 40, 1]
Layer name: conv1, layer type: QConv2D, input shapes: [[None, 50, 40, 1]], output shape: [None, 50, 40, 1]
Layer name: relu1, layer type: Activation, input shapes: [[None, 50, 40, 1]], output shape: [None, 50, 40, 1]
Layer name: pool1, layer type: AveragePooling2D, input shapes: [[None, 50, 40, 1]], output shape: [None, 25, 20, 1]
Layer name: conv2, layer type: QConv2D, input shapes: [[None, 25, 20, 1]], output shape: [None, 25, 20, 1]
Layer name: relu2, layer type: Activation, input shapes: [[None, 25, 20, 1]], output shape: [None, 25, 20, 1]
Layer name: pool2, layer type: AveragePooling2D, input shapes: [[None, 25, 20, 1]], output shape: [None, 5, 4, 1]
Layer name: flatten, layer type: Reshape, input shapes: [[None, 5, 4, 1]], output shape: [None, 20]
Layer name: dense1, layer type: QDense, input shapes: [[None, 20]], output shape: [None, 1]
Creati

In [ ]:
Y_hls_pred = hls_model.predict(X_test)
print("Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_hls_pred, axis=1))))

In [ ]:
plt.figure(figsize=(4, 4))
plot_roc(Y_test, Y_hls_pred, ['g','q','w','z','t'])

In [ ]:
del hls_model, config

# test bench

In [ ]:
X_test.shape

In [5]:
np.random.seed(42)
arr = np.zeros((3, 50, 40, 1))

In [6]:
arr[0, 3, 3, 0] = 1
arr[0, 4, 8, 0] = 2
arr[0, 7, 4, 0] = 3

arr[1, 4, 4, 0] = 1
arr[1, 4, 5, 0] = 2
arr[1, 2, 8, 0] = 3
arr[1, 3, 8, 0] = 4
arr[1, 5, 8, 0] = 5

arr[2, 4, 4, 0] = 1
arr[2, 5, 4, 0] = 2
arr[2, 6, 4, 0] = 3
arr[2, 3, 5, 0] = 4
arr[2, 2, 5, 0] = 5
arr[2, 5, 8, 0] = 6


In [7]:
for k in range(3):
  for i in range(50):
      row = arr[k, i, :, 0]
      row_str = ' '.join("_" if val == 0 else f"{val:1.0f}" for val in row)
      print(row_str)
  print('')

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ 1 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ 2 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ 3 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

In [ ]:
Y_arr = model.predict(arr)
Y_arr

In [ ]:
Y_hls_arr = hls_model.predict(arr)
Y_hls_arr

In [8]:
with open('model_test/tb_data/tb_input_features.dat', 'w') as my_file:
    for sample in arr.reshape(-1, np.prod(arr.shape[1:])):
        my_file.write(' '.join(str(x) for x in sample))
        my_file.write('\n')

with open('model_test/tb_data/tb_output_predictions.dat', 'w') as my_file:
    for sample in arr.reshape(-1, np.prod(arr.shape[1:])):
        my_file.write(' '.join(str(x) for x in sample))
        my_file.write('\n')